# Automatización de consultas sql

## librerias

In [1]:
import cx_Oracle
from prettytable import PrettyTable
from prettytable.colortable import RESET_CODE, ColorTable, Theme

## Datos de acceso

In [2]:
username = 'dstreamsdb'
password = 'claro2015'
dsn = 'exa1-scan.claro.amx/RAC8.world'
port = 1512
encoding = 'UTF-8'

## Consulta de base datos

In [63]:
def oraclesSentence(dbSentece):
    ### error no poner sus columnas, te va mostrar una sola columna
    """
          ** DESA **
        Host: exa1-scan.claro.amx
        Nombre de Servicio: RAC8.WORLD
        User: dstreamsdb
        Pass: claro2015
    """
    connection = None
    try: 
        connection = cx_Oracle.connect(username,password,dsn,encoding=encoding)
        ### obtenemos los campos para tener las columnas de las tablas, separando los campos de select por espacio que hay entre ambos 
        ### y lo cierra el espacio entre el ultimo campo y el from
        campos = dbSentece.split(" ")[1]
        tabla = dbSentece.split(" ")[3]
        ### en caso de un where ingresemos con #el valor string para cambiarlos por '' #
        new_s = dbSentece.replace("#","'")
        if campos == '*':
            columnas = []
            cursor = connection.cursor()
            for row in cursor.execute("SELECT column_name FROM all_tab_columns WHERE table_name = '"+tabla+"'"):                
                columnas.append(row[0])            
            cursor.close()            
            y = PrettyTable(columnas)
        else:
            ### Convertimos una lista la cadena de texto
            output=campos.split(',')  
            y = PrettyTable(output)
            
        cursor = connection.cursor()
        for fila in cursor.execute(new_s):
                y.add_row(fila)
                print()
        cursor.close()
        print(y.get_string(title="Resultado de "+tabla))
    except cx_Oracle.Error as error:
        print(error)
    finally:
    # release the connection
        if connection:
            connection.close()   

## Lectura de archivo

In [64]:
def getIDfromLine(line):
    ##regresame la sentencia sql que esta entre comillas simples
    return line.split("'")[1]

def readFileInput():
    inputfile = open("db.txt", 'r')
    lines = inputfile.readlines()
    for line in lines:
        readEscenario(line)

def readEscenario(line):
    if "FEATURE" in line.upper():
        display("FEATURE "+getIDfromLine(line))
    elif "SCENARIO" in line.upper():
        display("SCENARIO "+getIDfromLine(line))
    elif "CONSULT " in line.upper():
        
        dbSentece = getIDfromLine(line)        
        resultado = oraclesSentence(dbSentece)
        display(resultado)
        
readFileInput()    

'FEATURE Pruebas de cluspac'

'SCENARIO Se espera ejecuciÃ³n exitosa'

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                             Resultado de NEWNET_DB_DUMP_AR                                                                                            |
+-----------------+-----------------+---------------+-------------------+-------------------+-------------------+----------------+--------------+---------------------+-----------------+------------+------------------+
| NDB_SUBMIT_DATE | NDB_SUBMIT_TIME | NDB_RECIPIENT | NDB_RECIPIENT_CDR | NDB_DELIVERY_DATE | NDB_DELIVERY_TIME | NDB_ORIGINATOR | NDB_APP_NAME | NDB_DELIVERY_RESULT | NDB_SMS_CONTENT | NDB_MSG_ID | NDB_PROCESS_DATE |
+-----------------+-----------------+---------------+-------------------+-------------------+-------------------+---------------

None